# Data preparation

## Download data from RDM

In [1]:
!cp /QRISdata/Q0748/data/2023-2016-Jonathan-Prostate/2023-2016-Jonathan-Prostate-Bids.zip .
!unzip -q 2023-2016-Jonathan-Prostate-Bids.zip
!mv 2023-2016-Jonathan-Prostate-Bids bids-new
!rm 2023-2016-Jonathan-Prostate-Bids.zip

^C


## QSM Reconstruction

In [ ]:
!run_2_qsm.py bids-new qsm-new --premade body --auto_yes --slurm a_barth general

In [ ]:
!for f in `find qsm-new/qsm_final -name '*.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; cp "${f}" "bids-new/${sub}/${ses}/extra_data/"; done

In [ ]:
for f in `find . -wholename '*numpy_nibabel_twopass/mapflow/*/*echo-02*twopass.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; f2="${f/twopass1/twopass3}"; f2="${f2/echo-02/echo-04}"; ~/qsmxt/interfaces/nipype_interface_nonzeroaverage.py "${f}" "${f2}" "bids-new/${sub}/${ses}/extra_data/${sub}_${ses}_qsm_echo2-and-echo4.nii"; done

## Clean up segmentations

In [1]:
import os
import glob
import nibabel as nib
import numpy as np
import enum
import json
import scipy

In [2]:
bids_dir = "bids-new"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] == "t2starME_qsm_tra_Iso1.4mm_INPHASE_bipolar_RUN_THIS_ONE":
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))
print(f"{len(session_dirs)} sessions found:")
for session_dir in session_dirs: print(session_dir)

15 sessions found:
bids-new/sub-z0034542/ses-20220715
bids-new/sub-z0186251/ses-20221107
bids-new/sub-z0705200/ses-20230104
bids-new/sub-z0755228/ses-20211108
bids-new/sub-z1167038/ses-20220315
bids-new/sub-z1181657/ses-20220315
bids-new/sub-z1262112/ses-20220314
bids-new/sub-z1472355/ses-20221222
bids-new/sub-z1728751/ses-20220328
bids-new/sub-z1778013/ses-20220715
bids-new/sub-z1818796/ses-20230313
bids-new/sub-z2007565/ses-20220715
bids-new/sub-z2904752/ses-20220826
bids-new/sub-z3171177/ses-20230313
bids-new/sub-z3278008/ses-20211109


In [4]:
qsm_files = [glob.glob(os.path.join(session_dir, "extra_data", "*echo2-and-echo4.nii*")) for session_dir in session_dirs]
seg_files = [glob.glob(os.path.join(session_dir, "extra_data", "*segmentation.*")) for session_dir in session_dirs]

qsm_files = [qsm_file_list[0] for qsm_file_list in qsm_files if len(qsm_file_list)]
seg_files = [seg_file_list[0] for seg_file_list in seg_files if len(seg_file_list)]

print(f"{len(qsm_files)} QSM images found.")
print(f"{len(seg_files)} segmentations found.")

class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(seg_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # separate prostate tissue values and then remove segmented area
    prostate_values = qsm[seg == SegType.PROSTATE.value]
    seg[seg == SegType.PROSTATE.value] = 0

    # identify susceptibility values less than two standard deviations below the mean
    inliers = np.logical_and(np.mean(prostate_values) - np.std(prostate_values)*2 < qsm, qsm < np.mean(prostate_values) + np.std(prostate_values) * 2)

    # clean up segmentations such that 'inliers' are excluded
    seg[np.logical_and(seg == SegType.GOLD_SEED.value, inliers)] = 0
    seg[np.logical_and(seg == SegType.GOLD_SEED.value, qsm > 0)] = 0
    seg[seg == SegType.GOLD_SEED.value] = 1 # NEW VALUE

    # create new custom calcification label    
    seg[seg == SegType.CALCIFICATION.value] = 0
    calc_mask = np.logical_and(seg == SegType.NO_LABEL.value, qsm < np.mean(prostate_values) - 4*np.std(prostate_values))
    calc_mask = scipy.ndimage.binary_opening(calc_mask)
    seg[calc_mask] = 2 # NEW VALUE


    # save result using original file extension
    extension = ".".join(seg_files[i].split('.')[1:])
    filename=f"{seg_files[i].split('.')[0]}_clean.{extension}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

15 QSM images found.
15 segmentations found.
Saved bids-new/sub-z0034542/ses-20220715/extra_data/sub-z0034542_ses-20220715_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0186251/ses-20221107/extra_data/sub-z0186251_ses-20221107_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0705200/ses-20230104/extra_data/sub-z0705200_ses-20230104_run-01_segmentation_clean.nii
Saved bids-new/sub-z0755228/ses-20211108/extra_data/sub-z0755228_ses-20211108_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z1167038/ses-20220315/extra_data/sub-z1167038_ses-20220315_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z1181657/ses-20220315/extra_data/sub-z1181657_ses-20220315_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z1262112/ses-20220314/extra_data/sub-z1262112_ses-20220314_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z1472355/ses-20221222/extra_data/sub-z1472355_ses-20221222_run-01_segmentation_clean.nii
Saved bids-new/sub-z1728751/ses-20220328/extra_data/sub-z1728751_ses-2022